In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [67]:
dfs = [pd.read_csv(f'Data/{i}/merged_movies_data_{i}.csv') for i in range(2005, 2025)]
df = pd.concat(dfs, ignore_index=True)
df.columns

Index(['Title', 'Movie Link', 'Year', 'Duration', 'MPA', 'Rating', 'Votes',
       'budget', 'grossWorldWide', 'gross_US_Canada', 'opening_weekend_Gross',
       'directors', 'writers', 'stars', 'genres', 'countries_origin',
       'filming_locations', 'production_companies', 'Languages', 'wins',
       'nominations', 'oscars', 'release_date'],
      dtype='object')

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10700 entries, 0 to 10699
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  10700 non-null  object 
 1   Movie Link             10700 non-null  object 
 2   Year                   10700 non-null  int64  
 3   Duration               10672 non-null  object 
 4   MPA                    9087 non-null   object 
 5   Rating                 10659 non-null  float64
 6   Votes                  10659 non-null  object 
 7   budget                 4749 non-null   float64
 8   grossWorldWide         9812 non-null   float64
 9   gross_US_Canada        9394 non-null   float64
 10  opening_weekend_Gross  8901 non-null   float64
 11  directors              10700 non-null  object 
 12  writers                10700 non-null  object 
 13  stars                  10700 non-null  object 
 14  genres                 10700 non-null  object 
 15  co

In [18]:
df.describe()

,Year,Rating,budget,grossWorldWide,gross_US_Canada,opening_weekend_Gross,wins,nominations,oscars,release_date
count,10700.000000,10659.000000,4.749000e+03,9.812000e+03,9.394000e+03,8.901000e+03,10700.0,10700.000000,10700.000000,10420.000000
mean,2014.654766,6.372146,1.688698e+08,4.674242e+07,1.864523e+07,5.951319e+06,0.0,10.948598,0.099252,2014.727831
std,5.828472,1.060177,4.490761e+09,1.469510e+08,5.638952e+07,1.822706e+07,0.0,28.736221,0.500172,5.887646
min,2005.000000,1.100000,1.000000e+00,9.500000e+01,8.000000e+01,1.100000e+01,0.0,0.000000,0.000000,2005.000000
25%,2009.000000,5.800000,4.000000e+06,1.484510e+05,5.522200e+04,1.116400e+04,0.0,0.000000,0.000000,2009.000000
50%,2015.000000,6.500000,1.500000e+07,1.859422e+06,3.506135e+05,6.086000e+04,0.0,0.000000,0.000000,2015.000000
75%,2020.000000,7.100000,4.300000e+07,2.294555e+07,7.635952e+06,2.161480e+06,0.0,10.000000,0.000000,2020.000000
max,2024.000000,9.600000,3.000000e+11,2.923706e+09,9.366622e+08,3.571150e+08,0.0,433.000000,10.000000,2024.000000


In [19]:
df.to_csv('Data/merged_movies_data.csv', index=False)

In [20]:
# 3. 결측치 및 이상치 탐색
# - 결측치 탐색
df.isnull().sum()

Title                       0
Movie Link                  0
Year                        0
Duration                   28
MPA                      1613
Rating                     41
Votes                      41
budget                   5951
grossWorldWide            888
gross_US_Canada          1306
opening_weekend_Gross    1799
directors                   0
writers                     0
stars                       0
genres                      0
countries_origin            0
filming_locations           0
production_companies        0
Languages                   0
wins                        0
nominations                 0
oscars                      0
release_date              280
dtype: int64

In [21]:
df['MPA'].value_counts()

MPA
R            3304
PG-13        2135
Not Rated    1814
PG            963
Unrated       364
TV-MA         169
TV-14         114
G              97
TV-PG          69
TV-G           25
NC-17          11
TV-Y7           5
16+             4
Approved        3
18+             3
13+             3
MA-17           1
M               1
TV-Y7-FV        1
TV-Y            1
Name: count, dtype: int64

In [22]:
# Duration 중에 h와 m이 존재하지 않는 row 확인
# 해당 row의 Duration을 MPA로 변경
for i in df[df['Duration'].str.contains('h|m') == False].index:
    df.loc[i, 'MPA'] = df.loc[i, 'Duration']
    df.loc[i, 'Duration'] = np.nan

In [23]:
# 데이터 전처리 (공통) - 정제 - 결측치 처리
# duration = "2h 30m" , " 30m", "2h"
def str_to_minutes(duration):
    if pd.isnull(duration):
        return duration
    if 'h' in duration:
        print(duration)
        duration = duration.split('h')
        hours = int(duration[0])
        if 'm' in duration[1]:
            minutes = int(duration[1].split('m')[0])
        else:
            minutes = 0
        return hours*60 + minutes
    elif 'm' in duration:
        return int(duration.split('m')[0])
    else:
        return duration
df = df.drop('Movie Link', axis=1)
df['MPA'] = df['MPA'].fillna("Not Rated")
df['Duration'].unique()
df['Duration'] = df['Duration'].apply(str_to_minutes)
df['Duration'] = df['Duration'].fillna(df['Duration'].mean())
df

2h 20m
2h 23m
2h 37m
1h 57m
3h 7m
1h 59m
2h 20m
1h 55m
1h 26m
2h
1h 58m
1h 53m
1h 46m
1h 21m
1h 31m
2h 16m
1h 35m
1h 30m
1h 56m
1h 38m
1h 33m
2h 14m
1h 41m
1h 35m
1h 34m
1h 44m
1h 20m
1h 50m
2h 1m
1h 59m
1h 49m
2h 4m
2h 8m
2h 12m
1h 45m
2h 4m
2h 16m
1h 41m
1h 30m
1h 40m
1h 42m
2h 5m
2h 24m
1h 43m
1h 25m
2h 25m
1h 41m
1h 25m
1h 58m
1h 28m
1h 17m
1h 35m
1h 20m
1h 41m
1h 49m
2h 8m
1h 56m
1h 42m
1h 55m
1h 35m
1h 44m
2h 44m
2h 24m
1h 34m
1h 37m
1h 29m
2h 9m
1h 38m
1h 27m
1h 37m
1h 44m
1h 33m
1h 59m
2h 9m
1h 58m
1h 45m
1h 48m
2h 16m
1h 34m
1h 53m
2h 9m
2h 10m
1h 53m
1h 46m
1h 46m
1h 36m
2h 1m
1h 53m
1h 30m
1h 33m
1h 36m
1h 57m
1h 40m
1h 41m
2h 15m
1h 54m
1h 45m
1h 41m
2h 3m
1h 39m
1h 33m
1h 59m
1h 45m
1h 32m
1h 43m
1h 37m
1h 38m
2h 2m
2h 4m
2h 2m
1h 45m
1h 23m
1h 56m
1h 47m
1h 33m
1h 49m
1h 40m
1h 16m
2h 14m
1h 37m
1h 50m
1h 58m
2h 6m
1h 8m
1h 52m
1h 47m
1h 34m
1h 26m
1h 40m
1h 23m
1h 39m
1h 34m
1h 45m
2h
1h 37m
1h 46m
1h 30m
2h 15m
1h 36m
1h 42m
1h 51m
1h 48m
1h 43m
1h 52m
1h 47m
1h 43m
1h 

,Title,Year,Duration,MPA,Rating,Votes,budget,grossWorldWide,gross_US_Canada,opening_weekend_Gross,...,stars,genres,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars,release_date
0,1. Star Wars: Episode III - Revenge of the Sith,2005,140.0,PG-13,7.6,867K,113000000.0,850035635.0,380270577.0,108435841.0,...,"['Hayden Christensen', 'Natalie Portman', 'Ewa...","['Action Epic', 'Adventure Epic', 'Dark Fantas...","['United States', 'Italy', 'Switzerland', 'Tha...","['Mount Etna, Catania, Sicily, Italy']","['Lucasfilm', 'Mestiere Cinema', 'Pandora Films']",['English'],0,64,1,2005.0
1,"2. The Chronicles of Narnia: The Lion, the Wit...",2005,143.0,PG,6.9,438K,180000000.0,745013115.0,291710957.0,65556312.0,...,"['Tilda Swinton', 'Georgie Henley', 'William M...","['Fantasy Epic', 'Supernatural Fantasy', 'Swor...","['United Kingdom', 'United States']","['Adrspach National Park, Trutnov, Czech Repub...","['Walt Disney Pictures', 'Walden Media']","['English', 'German', 'Brazilian Sign Language']",0,46,0,2005.0
2,3. Harry Potter and the Goblet of Fire,2005,157.0,PG-13,7.7,705K,150000000.0,897468952.0,290469928.0,102685961.0,...,"['Daniel Radcliffe', 'Emma Watson', 'Rupert Gr...","['Sword & Sorcery', 'Teen Adventure', 'Teen Fa...","['United Kingdom', 'United States']","['Glenfinnan Viaduct, Fort William, Highland, ...","['Warner Bros.', 'Heyday Films', 'Patalex Prod...","['English', 'French', 'Latin']",0,48,1,2005.0
3,4. War of the Worlds,2005,117.0,PG-13,6.5,485K,132000000.0,603873504.0,234280354.0,64878725.0,...,"['Tom Cruise', 'Dakota Fanning', 'Tim Robbins']","['Action Epic', 'Adventure Epic', 'Alien Invas...",['United States'],"['JF Kennedy Blvd., Bayonne, New Jersey, USA']","['Paramount Pictures', 'Dreamworks Pictures', ...",['English'],0,49,3,2005.0
4,5. King Kong,2005,187.0,PG-13,7.2,455K,207000000.0,556906378.0,218080025.0,50130145.0,...,"['Naomi Watts', 'Jack Black', 'Adrien Brody']","['Adventure Epic', 'Dark Romance', 'Dinosaur A...","['United States', 'New Zealand', 'Germany']","['Shelly Bay, Wellington, New Zealand']","['Universal Pictures', 'WingNut Films', 'Big P...",['English'],0,104,0,2005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10695,596. Vettaiyan,2024,163.0,Not Rated,7.1,38K,NaN,2397623.0,NaN,NaN,...,"['Fahadh Faasil', 'Amitabh Bachchan', 'Rajinik...","['Cop Drama', 'One-Person Army Action', 'Actio...",['India'],[],['Lyca Productions'],['Tamil'],0,0,0,2024.0
10696,597. Nr. 24,2024,111.0,Not Rated,7.7,3.4K,NaN,3301976.0,NaN,NaN,...,"['Mark Noble', 'Lisa Loven Kongsli', 'August W...","['Docudrama', 'Epic', 'Period Drama', 'Biograp...",['Norway'],"['Rjukan, Tinn, Norway']","['SF Norge A/S', 'Motion Blur Films']",['Norwegian'],0,0,0,2024.0
10697,598. Parallel,2024,88.0,Not Rated,5.3,1.7K,NaN,NaN,NaN,NaN,...,"['Danielle Deadwyler', 'Aldis Hodge', 'Edwin H...","['Drama', 'Sci-Fi', 'Thriller']",['United States'],[],"['Rumble Riot Pictures', 'Hodge Brothers Produ...",[],0,0,0,2024.0
10698,599. Fight or Flight,2024,166.0,Not Rated,7.0,117,NaN,NaN,NaN,NaN,...,"['Josh Hartnett', 'Katee Sackhoff', 'Charithra...","['Action', 'Comedy']","['United States', 'United Kingdom']",['Hungary'],"['Asbury Park Productions', 'The Royal Budapes...",['English'],0,0,0,2024.0


In [ ]:
df['Rating'] = df['Rating'].fillna(5.5)
# 1K = 1000 and 1M = 1000000 and 1.4K = 1400
def str_to_number(votes):
    if pd.isnull(votes):
        return votes
    if 'K' in votes:
        return float(votes[:-1])*1000
    elif 'M' in votes:
        return float(votes[:-1])*1000000
    else:
        return float(votes)
df['Votes'] = df['Votes'].apply(str_to_number)
df['Votes'] = df['Votes'].fillna(0)

In [33]:
df['grossWorldWide'] = df['grossWorldWide'].fillna(0)
df['gross_US_Canada'] = df['gross_US_Canada'].fillna(0)
df['opening_weekend_Gross'] = df['opening_weekend_Gross'].fillna(0)

In [ ]:
df = df.drop('release_date', axis=1)

In [39]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10700 entries, 0 to 10699
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  10700 non-null  object 
 1   Year                   10700 non-null  int64  
 2   Duration               10700 non-null  float64
 3   MPA                    10700 non-null  object 
 4   Rating                 10700 non-null  float64
 5   Votes                  10700 non-null  float64
 6   budget                 4749 non-null   float64
 7   grossWorldWide         10700 non-null  float64
 8   gross_US_Canada        10700 non-null  float64
 9   opening_weekend_Gross  10700 non-null  float64
 10  directors              10700 non-null  object 
 11  writers                10700 non-null  object 
 12  stars                  10700 non-null  object 
 13  genres                 10700 non-null  object 
 14  countries_origin       10700 non-null  object 
 15  fi

,Year,Duration,Rating,Votes,budget,grossWorldWide,gross_US_Canada,opening_weekend_Gross,wins,nominations,oscars,gross_US_anada
count,10700.000000,10700.000000,10700.000000,1.070000e+04,4.749000e+03,1.070000e+04,1.070000e+04,1.070000e+04,10700.0,10700.000000,10700.000000,1.070000e+04
mean,2014.654766,106.090278,6.368804,5.426478e+04,1.688698e+08,4.286324e+07,1.636946e+07,4.950719e+06,0.0,10.948598,0.099252,1.636946e+07
std,5.828472,20.382064,1.059514,1.370121e+05,4.490761e+09,1.413102e+08,5.318728e+07,1.677251e+07,0.0,28.736221,0.500172,5.318728e+07
min,2005.000000,44.000000,1.100000,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000e+00
25%,2009.000000,92.000000,5.800000,1.400000e+03,4.000000e+06,7.006450e+04,2.664550e+04,4.458250e+03,0.0,0.000000,0.000000,2.664550e+04
50%,2015.000000,102.000000,6.500000,7.900000e+03,1.500000e+07,1.200102e+06,2.042350e+05,2.940000e+04,0.0,0.000000,0.000000,2.042350e+05
75%,2020.000000,116.000000,7.100000,4.200000e+04,4.300000e+07,1.830332e+07,4.197936e+06,6.802742e+05,0.0,10.000000,0.000000,4.197936e+06
max,2024.000000,325.000000,9.600000,3.000000e+06,3.000000e+11,2.923706e+09,9.366622e+08,3.571150e+08,0.0,433.000000,10.000000,9.366622e+08


In [66]:
df[df['budget'].isnull()]['genres'].value_counts()
# 장르가 없는 데이터의 budget 확인
# df[df['genres'] == '[]']['budget'].value_counts()
df[df.budget.isnull()].genres.value_counts()

genres
['Documentary']                                                                          618
['Drama']                                                                                528
[]                                                                                       310
['Comedy', 'Drama']                                                                      200
['Drama', 'Romance']                                                                     171
                                                                                        ... 
['Dystopian Sci-Fi', 'Horror', 'Sci-Fi', 'Thriller']                                       1
['Gun Fu', 'Martial Arts', 'Action', 'Crime', 'Thriller']                                  1
['B-Action', 'Action', 'Crime']                                                            1
['Dark Comedy', 'Action', 'Comedy', 'Crime']                                               1
['Adult Animation', 'Superhero', 'Action', 'Animation', 'Drama'